In [ ]:
import sys
sys.path.insert(0, '../BO')

from gp import *
import numpy as np
from sklearn.datasets import fetch_mldata
from LG import *# Load and format the training and testing data

mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

loss = train_logisitc_regression_(mnist)

In [ ]:
# set parameters
bounds = np.array([[-15., 0.0], [5, 200], [20, 2000], [0, 1]])
# maxeps specifies how many BOs we want to run
maxeps = 10
# n_iters is number of iterations we want for each run of BO
n_iters = 100
acqui_eva_num = 5
n_pre_samples = 3
greater_is_better = False
y_list = list()
coor_sigma = 1 * np.array([0.2,0.2,0.2,0.2,0.2,0.2])
burn_in = 10
input_dimension = 4
mode = 'MAP'
acqui_mode = 'MCMC'
acqui_sample_num = 5
process_sample_mode = 'rho'
prior_mode ='normal_prior'
likelihood_mode = 'rho_likelihood'

In [ ]:
y_list = list()

for i in range(n_iters):
    y_list.append(list())


for j in range(maxeps):
    print ('Running %d episode' % (j + 1))
    xp, yp = bayesian_optimisation(coor_sigma = coor_sigma, 
                                   burn_in = burn_in, 
                                   input_dimension = input_dimension,
                                   n_iters = n_iters, 
                                   sample_loss = loss, 
                                   bounds = bounds, 
                                   n_pre_samples = n_pre_samples, 
                                   acqui_eva_num = acqui_eva_num,  
                                   greater_is_better=greater_is_better, 
                                   mode = mode, 
                                   acqui_mode = acqui_mode, 
                                   acqui_sample_num = acqui_sample_num,   
                                   process_sample_mode = process_sample_mode, 
                                   prior_mode = prior_mode, 
                                   likelihood_mode = likelihood_mode)
    y_train = yp[n_pre_samples:]
    for idx in range(len(y_train)):
        if idx == 0:
            y_list[idx].append(y_train[idx])
        else:
            y_list[idx].append(np.min(y_train[:idx]))

In [ ]:
mean_list, std_list = return_mean_std(y_list)

In [ ]:
import matplotlib.pyplot as plt
x = [i+1 for i in range(len(y_list))]

plt.errorbar(x, mean_list, yerr = std_list, fmt = '-')

plt.show()

In [ ]:
import pickle
opt_dict = {}
opt_dict['y_list'] = y_list
with open('LG_MCMC_RHO.pkl','wb') as file:
    pickle.dump(opt_dict, file)

In [ ]:
with open('LG_MCMC_RHO.pkl', 'rb') as pickle_file:
    data1 = pickle.load(pickle_file)